In [176]:
import pandas as pd
import numpy as np
import ast
from sklearn.cluster import KMeans, MiniBatchKMeans

In [177]:
filename = '../data/2_embeddings_raw_data.csv'

# Read the TSV file into a pandas DataFrame
df = pd.read_csv(filename, on_bad_lines='warn')

In [178]:
df['Embedding'] = df['Embedding'].apply(ast.literal_eval)
df['Embedding'] = df['Embedding'].apply(lambda x: [float(num) for num in x])

df

,auid,name,EthnicSeer,prop,lastname,firstname,Ethnea,Genni,SexMac,SSNgender,Highest_probF_ethnicity,Highest_probF_value,Embedding
0,9731334_2,Cameron 'Dale' Bass,ITA,0.653567,'Dale' Bass,Cameron,ENGLISH,M,mostly_male,-,ENGLISH,92.191,"[0.0027535639237612486, -0.006899471394717693,..."
1,2155715_1,Bert Hart,ENG,0.772359,Hart,Bert,DUTCH,M,male,M,DUTCH,87.200,"[-0.012196633964776993, -0.034759119153022766,..."
2,14609221_2,Esther Nolte- Hoen,GER,0.665081,Nolte- Hoen,Esther,GERMAN,F,female,F,HISPANIC,43.243,"[-0.02559061162173748, -0.02379501983523369, -..."
3,8101337_1,Ellen 't Hoen,CHI,0.665526,'t Hoen,Ellen,DUTCH,F,female,F,DUTCH,37.459,"[-0.014605682343244553, -0.030205124989151955,..."
4,9804785_2,Peter 't Hoen,GER,0.330864,'t Hoen,Peter,DUTCH,M,male,M,GERMAN,34.203,"[0.018846435472369194, -0.026805326342582703, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35887,9239360_5,Senda Mezghani,ARA,0.691154,Mezghani,Senda,ARAB,F,andy,F,HISPANIC,60.288,"[0.0009060240699909627, -0.012940743006765842,..."
35888,9258893_4,Raquelle Mesholam-Gately,FRN,0.671148,Mesholam-Gately,Raquelle,ENGLISH-HISPANIC,F,andy,F,HISPANIC,99.978,"[-0.009965583682060242, 0.001120477681979537, ..."
35889,12621069_2,Niki Messini-Nikolaki,RUS,0.946401,Messini-Nikolaki,Niki,GREEK,F,female,F,GREEK,96.862,"[-0.00857141800224781, -0.01564069651067257, 0..."
35890,3154416_5,Tahar Mestiri,FRN,0.376373,Mestiri,Tahar,ARAB,M,male,-,ARAB,83.593,"[-0.012485790997743607, -0.01225669402629137, ..."


In [244]:
# Remove some names with low probability of being in top ethnicities to have better class balance for clustering
top_ethnicities = ['HISPANIC', 'ENGLISH', 'INDIAN', 'ARAB', 'CHINESE', 'JAPANESE', 'NORDIC', 'SLAV']
df = df[~df["Highest_probF_ethnicity"].isin(top_ethnicities) | ((df["Highest_probF_ethnicity"].isin(top_ethnicities) & (df["Highest_probF_value"] > 95)))]

In [256]:
# ethnea_counts = df['Highest_probF_ethnicity'].value_counts()
# print(ethnea_counts.to_string())
# print(len(ethnea_counts))

In [247]:
# Convert the 'Embedding' column to a numpy array
X = np.array(df['Embedding'].tolist())

# Normalize the embeddings
X_normalized = X / np.linalg.norm(X, axis=1, keepdims=True)

print(len(X))

21813


In [263]:
%%time
# Apply K-means clustering
df_copy = df.copy()
kmeans = MiniBatchKMeans(n_clusters=200, random_state=0, batch_size=2048)
kmeans.fit(X_normalized)

y_kmeans = kmeans.predict(X_normalized)
print(y_kmeans[0:10])
print(len(y_kmeans))


/Users/zara/miniforge3/envs/debias/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[ 97  41  98 111 121 135 185  52  96 136]
21813
CPU times: user 11min 28s, sys: 7.73 s, total: 11min 35s
Wall time: 1min 19s


In [270]:
# Add 'Cluster' column to the copied DataFrame
df_copy['Cluster'] = y_kmeans

# Sort DataFrame by 'Cluster' column
df_sorted = df_copy[['firstname', 'Cluster', 'Highest_probF_ethnicity', 'Highest_probF_value', 'Genni']].sort_values(by='Cluster')
# Rename columns
df_sorted.rename(columns={'Highest_probF_ethnicity': 'Ethnicity', 'Highest_probF_value': 'Ethnicity Probability', 'Genni': 'Gender'}, inplace=True)
df_sorted['Group'] = list(zip(df_sorted['Ethnicity'], df_sorted['Gender']))

# Save to csv
csv_filename = '../data/3_clusters_200.csv'
df_sorted.to_csv(csv_filename, index=False)
print(f"Sorted embeddings saved to {csv_filename}.")

Sorted embeddings saved to ../data/3_clusters_200.csv.


In [271]:
df_sorted

,firstname,Cluster,Ethnicity,Ethnicity Probability,Gender,Group
30514,Warwick,0,ENGLISH,95.829,M,"(ENGLISH, M)"
14923,Hastings,0,AFRICAN,77.214,M,"(AFRICAN, M)"
10917,York,0,GERMAN,85.464,M,"(GERMAN, M)"
7722,Stirling,0,ENGLISH,95.523,M,"(ENGLISH, M)"
20564,Albion,0,ROMANIAN,61.645,M,"(ROMANIAN, M)"
...,...,...,...,...,...,...
10270,Dongwon,199,KOREAN,96.171,M,"(KOREAN, M)"
18837,Daehee,199,KOREAN,97.545,M,"(KOREAN, M)"
18838,Dongmin,199,KOREAN,83.667,M,"(KOREAN, M)"
17054,Daekwan,199,KOREAN,87.916,M,"(KOREAN, M)"


In [272]:
## Creates a df with information about each cluster

cluster_sizes = df_sorted['Cluster'].value_counts().reset_index().rename(columns={'index': 'Cluster', 'Cluster': 'Size'})

# Find the group with the highest count for each cluster
highest_group = df_sorted.groupby('Cluster')['Group'].apply(lambda x: x.value_counts().idxmax()).reset_index(name='Group')

# Calculate the percentage of rows with the highest group for each cluster
highest_group_percent = df_sorted.groupby('Cluster')['Group'].apply(lambda x: (x.value_counts(normalize=True).max() * 100).round(2)).reset_index(name='Group_Acc')

# Merge the DataFrames
result_df = pd.merge(cluster_sizes, highest_group, on='Cluster')
result_df = pd.merge(result_df, highest_group_percent, on='Cluster')

result_df = result_df.sort_values(by='Group_Acc').reset_index(drop=True)

# Print the combined DataFrame
result_df


,Cluster,Size,Group,Group_Acc
0,113,223,"(TURKISH, F)",7.17
1,84,186,"(ISRAELI, F)",8.06
2,80,152,"(HISPANIC, M)",9.21
3,43,52,"(DUTCH, M)",9.62
4,167,82,"(GERMAN, M)",9.76
...,...,...,...,...
195,103,122,"(JAPANESE, M)",94.26
196,176,82,"(JAPANESE, M)",95.12
197,155,141,"(INDIAN, M)",99.29
198,112,20,"(KOREAN, M)",100.00


In [273]:
## Only takes clusters with high ethnicity/gender agreement

# Filter clusters with Ethnicity and Gender Acc > 50%
chosen_clusters = result_df[(result_df['Group_Acc'] > 50)]

# Only keep size of 10 or above
chosen_clusters = chosen_clusters[chosen_clusters['Size'] > 9]

# Print the filtered DataFrame
print(chosen_clusters.reset_index(drop=True).tail())
print("Average Cluster Size: " + str(chosen_clusters['Size'].mean()))

    Cluster  Size          Group  Group_Acc
76       42   145  (JAPANESE, F)      93.10
77      103   122  (JAPANESE, M)      94.26
78      176    82  (JAPANESE, M)      95.12
79      155   141    (INDIAN, M)      99.29
80      112    20    (KOREAN, M)     100.00
Average Cluster Size: 121.58024691358025


In [303]:
grouped_df = chosen_clusters.groupby(['Group'])
selected_rows_df = pd.DataFrame()

for group_id, (group_label, group_data) in enumerate(grouped_df):
    cluster_list = group_data['Cluster'].to_list()

    # Find all matching rows in 'results_df' with the same 'Group'
    # from a list of clusters
    matching_rows = df_sorted[
        (df_sorted['Group'] == group_label) &
        (df_sorted['Cluster'].isin(cluster_list))
    ]

    # Take a random 10 rows from the matching rows
    selected_rows = matching_rows.sample(n=10, random_state=32)

    # Concatenate the selected rows with the 'selected_rows_df' DataFrame
    selected_rows_df = pd.concat([selected_rows_df, selected_rows], ignore_index=True)


selected_rows_df

,firstname,Cluster,Ethnicity,Ethnicity Probability,Gender,Group
0,Oladipo,122,AFRICAN,94.404,M,"(AFRICAN, M)"
1,Adebowale,122,AFRICAN,99.733,M,"(AFRICAN, M)"
2,Adedeji,122,AFRICAN,92.914,M,"(AFRICAN, M)"
3,Olumide,122,AFRICAN,85.657,M,"(AFRICAN, M)"
4,Obafemi,122,AFRICAN,97.160,M,"(AFRICAN, M)"
...,...,...,...,...,...,...
325,Ozgen,76,TURKISH,99.991,M,"(TURKISH, M)"
326,Osman,76,TURKISH,96.523,M,"(TURKISH, M)"
327,Murat,76,TURKISH,98.830,M,"(TURKISH, M)"
328,Suleyman,76,TURKISH,99.263,M,"(TURKISH, M)"


In [304]:
# Find any missing F groups
missing_groups = [('AFRICAN', 'F'), ('BALTIC', 'F'), ('GERMAN', 'F'), ('HUNGARIAN', 'F'), 
                    ('ISRAELI', 'F'), ('ITALIAN', 'F'), ('NORDIC', 'F')]

unique_groups = list(chosen_clusters['Group'].unique())
unique_groups_sorted = sorted(unique_groups + missing_groups)

for group in unique_groups_sorted:
    print(group)

('AFRICAN', 'F')
('AFRICAN', 'M')
('ARAB', 'F')
('ARAB', 'M')
('BALTIC', 'F')
('BALTIC', 'M')
('CHINESE', 'F')
('CHINESE', 'M')
('DUTCH', 'F')
('DUTCH', 'M')
('ENGLISH', 'F')
('ENGLISH', 'M')
('FRENCH', 'F')
('FRENCH', 'M')
('GERMAN', 'F')
('GERMAN', 'M')
('GREEK', 'F')
('GREEK', 'M')
('HISPANIC', 'F')
('HISPANIC', 'M')
('HUNGARIAN', 'F')
('HUNGARIAN', 'M')
('INDIAN', 'F')
('INDIAN', 'M')
('ISRAELI', 'F')
('ISRAELI', 'M')
('ITALIAN', 'F')
('ITALIAN', 'M')
('JAPANESE', 'F')
('JAPANESE', 'M')
('KOREAN', 'F')
('KOREAN', 'M')
('NORDIC', 'F')
('NORDIC', 'M')
('SLAV', 'F')
('SLAV', 'M')
('THAI', 'F')
('THAI', 'M')
('TURKISH', 'F')
('TURKISH', 'M')


In [305]:
# Print 10 random first names from each ethnicity
for group in missing_groups:
    selected_rows = df_sorted[(df_sorted["Group"] == group) & (df_sorted['Ethnicity Probability'] > 90)].sample(n=10, replace=True)
    selected_rows_df = pd.concat([selected_rows_df, selected_rows], ignore_index=True)
    print(f"{group}: {selected_rows['firstname'].tolist()}")


('AFRICAN', 'F'): ['Adenike', 'Chinwe', 'Olayinka', 'Ramola', 'Eucharia', 'Njeri', 'Olukemi', 'Adwoa', 'Nkechi', 'Folasade']
('BALTIC', 'F'): ['Nijole', 'Snieguole', 'Ruta', 'Dace', 'Radvile', 'Audrone', 'Baiba', 'Laima', 'Skaidra', 'Vaida']
('GERMAN', 'F'): ['Etheresia', 'Rosmarie', 'Wibke', 'Waltraud', 'Ludgera', 'Brigitta', 'Karola', 'Gerhild', 'Heidemaria', 'Annegret']
('HUNGARIAN', 'F'): ['Katelin', 'Erzsebet', 'Ibolya', 'Agota', 'Sarolta', 'Zsofia', 'Eniko', 'Gyongyi', 'Erzsebet', 'Erzsebet']
('ISRAELI', 'F'): ['Maayan', 'Tziporah', 'Dganit', 'Tamy', 'Ayelet', 'Anat', 'Raya', 'Ifat', 'Liora', 'Hefziba']
('ITALIAN', 'F'): ['Loriana', 'Annagrazia', 'Orietta', 'Lanna', 'Samuela', 'Tambra', 'Ausilia', 'Cherita', 'Concetta', 'Donatella']
('NORDIC', 'F'): ['Kirsi', 'Yrsa', 'Anni', 'Jaana', 'Unni', 'Kirstine', 'Guro', 'Anjeli', 'Holmfridur', 'Liv']


In [306]:
selected_rows_df.to_csv('../data/4c_name_groups.csv')

In [207]:
# # OPTIONAL: Chart showing gender and ethnicity agreement

# ## Creates a df_sorted with information about each cluster

# cluster_sizes = df_sorted['Cluster'].value_counts().reset_index().rename(columns={'index': 'Cluster', 'Cluster': 'Size'})

# # Calculate the percentage of rows with the highest ethnicity for each cluster
# highest_ethnicity_percent = df_sorted.groupby('Cluster')['Ethnicity'].apply(lambda x: (x.value_counts(normalize=True).max() * 100).round(2)).reset_index(name='Ethnicity_Acc')

# # Calculate the percentage of rows with the highest gender for each cluster
# highest_gender_percent = df_sorted.groupby('Cluster')['Gender'].apply(lambda x: (x.value_counts(normalize=True).max() * 100).round(2)).reset_index(name='Gender_Acc')

# # Find the ethnicity with the highest count for each cluster
# highest_ethnicity = df_sorted.groupby('Cluster')['Ethnicity'].apply(lambda x: x.value_counts().idxmax()).reset_index(name='Ethnicity')

# # Find the gender with the highest count for each cluster
# highest_gender = df_sorted.groupby('Cluster')['Gender'].apply(lambda x: x.value_counts().idxmax()).reset_index(name='Genni')

# # Merge the DataFrames
# result_df = pd.merge(cluster_sizes, highest_ethnicity, on='Cluster')
# result_df = pd.merge(result_df, highest_ethnicity_percent, on='Cluster')
# result_df = pd.merge(result_df, highest_gender, on='Cluster')
# result_df = pd.merge(result_df, highest_gender_percent, on='Cluster')

# result_df = result_df.sort_values(by='Cluster').reset_index(drop=True)

# import matplotlib.pyplot as plt

# plt.figure(figsize=(8, 6))  # Set the figure size

# # Scatter plot
# plt.scatter(result_df['Ethnicity_Acc'], result_df['Gender_Acc'])

# # Add labels to the points
# for i in range(len(result_df)):
#     plt.text(result_df['Ethnicity_Acc'][i], result_df['Gender_Acc'][i], result_df['Ethnicity'][i])

# # Set the axis labels
# plt.xlabel('Ethnicity Accuracy')
# plt.ylabel('Gender Accuracy')

# # Set the title
# plt.title('Accuracy by Ethnicity and Gender')

# # Show the plot
# plt.show()
